# Chess Engine with PyTorch

## Imports

In [1]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import random_split, DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from auxiliary_func import check_memory

2025-10-24 23:43:34.547385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-24 23:43:34.566494: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-24 23:43:34.566527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-24 23:43:34.578385: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 23:43:35.427009: W tensorflow/compiler/tf

# Data preprocessing

## Load data

In [ ]:
# Calcute memory distribution so that loading pgns is 10% of processed data, 1.5 gb leftover

total_mem = check_memory() - 4.0
pgn_memory_mark = total_mem - (total_mem/20) + 4.0
print(total_mem)
print(pgn_memory_mark)

8.071182250976562
11.264064025878906


In [ ]:
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        # print(file_path)
        i = 0
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)


            i += 1
            if i % 100 == 0:
                available_gb = check_memory()

                if available_gb < pgn_memory_mark:
                    print(f"Low memory hit")
                    return games
    return games

files = [file for file in os.listdir("../../data") if file.endswith(".pgn")]
# Sort by file size (ascending)
files_sorted = sorted(files, key=lambda f: os.path.getsize(os.path.join("../../data", f)))

LIMIT_OF_FILES = min(len(files), 30)
games = []
i = 1
for file in tqdm(files_sorted):
    games.extend(load_pgn(f"../../data/{file}"))
    # print(len(games))

    available_gb = check_memory()
    if i >= LIMIT_OF_FILES or available_gb < pgn_memory_mark:
        print(i)
        break
    i += 1

 32%|███▏      | 25/79 [00:48<01:45,  1.94s/it]

Low memory hit
26


In [4]:
print(f"GAMES PARSED: {len(games)}")
available_gb = check_memory()
print(f"Available Memory: {available_gb}")

GAMES PARSED: 17481
Available Memory: 11.259632110595703


## Convert data into tensors

In [5]:
from auxiliary_func import create_input_for_nn, encode_moves

In [6]:
X, y = create_input_for_nn(games)



 68%|██████▊   | 11900/17481 [01:43<00:48, 115.36it/s]


Low memory hit at 11900 games
4.963600158691406


In [7]:
print(f"NUMBER OF SAMPLES: {len(y)}")
available_gb = check_memory()
print(f"Available Memory: {available_gb}")

NUMBER OF SAMPLES: 975522
Available Memory: 1.95953369140625


In [8]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)
available_gb = check_memory()
print(f"Available Memory: {available_gb}")

Available Memory: 1.96820068359375


In [9]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Preliminary actions

In [10]:
from dataset import ChessDataset
from model import ChessModel

In [11]:
# Create Dataset
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Compute split sizes
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Then create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cuda


# Data Collection

In [12]:
# Get current time in a readable format
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")

# Create a unique log directory
log_dir = f"../../runs/experiment_i{len(y)}_{current_time}"

# Create the SummaryWriter
writer = SummaryWriter(log_dir=log_dir)

# Training

In [ ]:
num_epochs = 200
for epoch in range(num_epochs):
    start_time = time.time()

    # Training
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()


    # Stats
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)

    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60

    if epoch % 20 == 0:
        # Save the model
        torch.save(model.state_dict(), f"../../models/TORCH_{epoch}EPOCHS_random_data.pth")  
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, Time: {minutes}m{seconds}s')

    writer.add_scalar("Loss/train", avg_train_loss, epoch + 1)
    writer.add_scalar("Loss/validation", avg_val_loss, epoch + 1)

writer.close()

100%|██████████| 13719/13719 [00:51<00:00, 267.38it/s]


Epoch 1/200, Loss: 4.4058, Time: 0m52s


100%|██████████| 13719/13719 [00:50<00:00, 269.74it/s]


Epoch 2/200, Loss: 3.2896, Time: 0m52s


100%|██████████| 13719/13719 [00:53<00:00, 256.61it/s]


...

Epoch 198/200, Loss: 0.3398, Time: 0m55s


100%|██████████| 13719/13719 [00:54<00:00, 253.47it/s]


Epoch 199/200, Loss: 0.3379, Time: 0m55s


100%|██████████| 13719/13719 [00:53<00:00, 255.49it/s]


Epoch 200/200, Loss: 0.3376, Time: 0m55s


# Save the model and mapping

In [ ]:
# Save the model
torch.save(model.state_dict(), "../../models/TORCH_200EPOCHS_random_data.pth")

In [15]:
import pickle

with open("../../models/heavy_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)